# Notebook for extracting features for authorship attribution.
In this notebook all authorship attribution features will be collected. They will be saved after extraction. Make sure, you define each feature extraction in a function, so it easily can be repurposed.

Author: lkt259@alumni.ku.dk

In [27]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

#### Sentence length
Nice feature, very complex!

In [2]:
string = "This list has overlapping features with content features. For example, word n-grams will capture the content of the text along with stylometric tendencies. Content features consist of word frequencies, word and character n-grams, hapax legomena etc. This overlap is not of concern, however, as Sari et al. \cite{Sari2018} show, using content features is beneficial when performing authorship attribution of news articles because journalists often have certain topics they prefer writing about. They argue that using only stylometric features is beneficial when attributing authors to texts of the same topic or genre, e.g. law text or movie reviews."
test_corpus = ['This list has overlapping features with content features. For example, word n-grams will capture the content of the text along with stylometric tendencies. Content features consist of word frequencies, word and character n-grams, hapax legomena etc. This overlap is not of concern, however, as Sari et al. \cite{Sari2018} show, using content features is beneficial when performing authorship attribution of news articles because journalists often have certain topics they prefer writing about. They argue that using only stylometric features is beneficial when attributing authors to texts of the same topic or genre, e.g. law text or movie reviews.',
              'Bozkurt et al. \cite{Bozkurt} performed authorship attribution on Turkish newspaper articles using stylometric features, vocabulary diversity, bag of words and frequency of function words. For stylometric features, they used number of sentences and words in the article, the average number of words in a sentence and the whole article, the vocabulary size, frequencies of symbols used (periods, exclamation marks, etc.) and the number of incomplete sentences. They weighted their features using Term Frequency-Inverse Document Frequency (TF-IDF).',
              'TF-IDF is a weighting system often used on words. It consists of two parts: term frequency (TF), which coulds how often a term occurs in a document, and inverse document frequency (IDF) which measures the term importance, as it compares how often the term occurs in a corpus\cite{TFIDF}. The system is useful for measuring how important a term is for a document. For example, the word "the" will often occur in English texts, making it not important even though it occurs frequently in a document. TF-IDF has on many occasions been used in authorship attribution\cite{Muttenhaler, basile:2019, rahgouy:2019} and can be used on different terms, such as characters, symbols or n-grams.',
              'Every year, the research group Webis hosts the PAN shared tasks on digital text forensics and sylometry\cite{PAN}. Contestants will solve various tasks concerning NLP and on multiple occasions, authorship attribution were part of the tasks. The methodologies used in these tasks are useful resources for feature and model selections.',
              'In the overview paper of the authorship attribution task of PAN 2019\cite{kestemont2019overview}, the 12 best performing models are compiled. All features involve character n-grams and other sorts of n-grams. Other popular choices of n-grams contain words, POS tags, punctuation and distortion. Most of the participants use TF-IDF weighting and SVMs as classifiers. Distortion is a method for masking topical contents of the text before feature extraction, focusing only on stylometric features.']

In [34]:
def split_sentences(text):
    '''Returns an array with text split into sentences'''
    return np.array(re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text), dtype=str)

def remove_dots(word):
    return re.sub(r',|\.|:|!|\?|;', '', word)

def split_words(text):
    '''Returns an array with text split into words'''
    text = text.lower()
    string = np.array(text.split(), dtype=str)
    no_dot = np.array([remove_dots(x) for x in string])
    return no_dot

def get_sentence_lengths(text):
    '''Returns dictionary with sentence lengh in chars and words'''
    split_text = split_sentences(text)
    num_sentences = len(split_text)
    num_chars = np.array([len(x) for x in split_text], dtype=int)
    num_words = [split_words(x).size for x in split_sentences(string)]
    return {'chars' : num_chars, 'words' : num_words, 'num_sents' : num_sentences}

def get_sentence_length_stats(text):
    '''Returns dictionary with mean, std and median lengths in both chars and words'''
    sentence_lengths = get_sentence_lengths(text)
    output = {'number_of_sentences' : sentence_lengths['num_sents'],
              'avg_sent_len_chars' : np.mean(sentence_lengths['chars']),
              'std_sent_len_chars' : np.std(sentence_lengths['chars']),
              'med_sent_len_chars' : np.median(sentence_lengths['chars']),
              
              'avg_sent_len_words' : np.mean(sentence_lengths['words']),
              'std_sent_len_words' : np.std(sentence_lengths['words']),
              'med_sent_len_words' : np.median(sentence_lengths['words'])
             }
    return output

print("Stats for our test document")
get_sentence_length_stats(string)

Stats for our test document


{'number_of_sentences': 6,
 'avg_sent_len_chars': 107.33333333333333,
 'std_sent_len_chars': 48.65410796862093,
 'med_sent_len_chars': 95.0,
 'avg_sent_len_words': 16.166666666666668,
 'std_sent_len_words': 6.618576550554926,
 'med_sent_len_words': 14.0}

##### Word length
The count of words of the entire text.
Also extremely complex feature, cool shit.

In [4]:
def get_word_lengths(split_text):
    '''Returns length of words in characters'''
    return np.array([len(x) for x in split_text], dtype=int)

def get_word_length_stats(text):
    '''Returns various stats for words in document'''
    #Split text here, to reduce function calls.
    split_text = split_words(text)
    word_lengths = get_word_lengths(split_text)
    output = {
        'number_of_words' : len(split_text),
        'avg_word_len_chars' : np.mean(word_lengths),
        'std_word_len_chars' : np.std(word_lengths),
        'med_word_len_chars' : np.median(word_lengths)
    }
    return output

print("Test word lengths")
get_word_length_stats(string)

Test word lengths


{'number_of_words': 97,
 'avg_word_len_chars': 5.546391752577319,
 'std_word_len_chars': 2.853901719013402,
 'med_word_len_chars': 5.0}

### Word frequency
Get word frequencies with TF-IDF weightings.

In [19]:
def train_tfidf_word_freq(corpus):
    '''Trains a TF-IDF vectorizer on word frequencies'''
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    
    feature_names = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    display(df.head(2))
    
    return X, vectorizer 

def get_tfidf_word_freq(vectorizer, texts):
    '''Returns the TF-IDF weighted word frequencies of test documents'''
    #Multiple texts required
    return vectorizer.transform(texts)

X, vectorizer = train_tfidf_word_freq(test_corpus)

,12,2019,about,al,all,along,and,are,argue,article,...,were,when,which,whole,will,with,word,words,writing,year
0,0.0,0.0,0.091857,0.074110,0.0,0.091857,0.04377,0.0,0.091857,0.000000,...,0.0,0.183714,0.0,0.000000,0.061518,0.183714,0.222329,0.000000,0.091857,0.0
1,0.0,0.0,0.000000,0.083392,0.0,0.000000,0.19701,0.0,0.000000,0.206724,...,0.0,0.000000,0.0,0.103362,0.000000,0.000000,0.000000,0.276891,0.000000,0.0


### Hapax legomena
Count how many unique words are in a document.

In [38]:
nltk.probability.FreqDist(split_words(string))

FreqDist({'features': 5, 'of': 5, 'content': 4, 'word': 3, 'the': 3, 'is': 3, 'this': 2, 'with': 2, 'n-grams': 2, 'text': 2, ...})